# **📄 Document type classification baseline code**
> 문서 타입 분류 대회에 오신 여러분 환영합니다! 🎉     
> 아래 baseline에서는 ResNet 모델을 로드하여, 모델을 학습 및 예측 파일 생성하는 프로세스에 대해 알아보겠습니다.

## Contents
- Prepare Environments
- Import Library & Define Functions
- Hyper-parameters
- Load Data
- Train Model
- Inference & Save File


## 1. Prepare Environments

* 데이터 로드를 위한 구글 드라이브를 마운트합니다.
* 필요한 라이브러리를 설치합니다.

In [52]:
print(1570 * 8 * 8)

100480


In [1]:
# 구글 드라이브 마운트, Colab을 이용하지 않는다면 패스해도 됩니다.
#from google.colab import drive
#rive.mount('/gdrive', force_remount=True)
#drive.mount('/content/drive')

In [2]:
# 구글 드라이브에 업로드된 대회 데이터를 압축 해제하고 로컬에 저장합니다.
#!tar -xvf drive/MyDrive/datasets_fin.tar > /dev/null

In [3]:
# !pip install -r requirements.txt 이미 서버에 설치되어 있음

In [4]:
# 필요한 라이브러리를 설치합니다.
# !pip install timm 이미 서버에 설치되어 있음

## 2. Import Library & Define Functions
* 학습 및 추론에 필요한 라이브러리를 로드합니다.
* 학습 및 추론에 필요한 함수와 클래스를 정의합니다.

In [5]:
import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

### 추가내용
from datetime import datetime       # 폴더이름에 날짜시간을 넣기 위한 라이브러리


In [6]:
# 시드를 고정합니다.
SEED = 42  # 재현 가능한 결과를 위해 사용할 시드 값 설정

# 파이썬의 해시 시드를 고정하여 파이썬의 해시 값이 일정하도록 설정
os.environ['PYTHONHASHSEED'] = str(SEED)

# 파이썬의 랜덤 시드를 고정하여 랜덤 연산이 재현 가능하도록 설정
random.seed(SEED)

# Numpy의 랜덤 시드를 고정하여 Numpy 연산이 재현 가능하도록 설정
np.random.seed(SEED)

# PyTorch의 랜덤 시드를 고정하여 PyTorch 연산이 재현 가능하도록 설정
torch.manual_seed(SEED)

# CUDA 사용 시 각 디바이스의 랜덤 시드를 고정하여 CUDA 연산이 재현 가능하도록 설정
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)  # 여러 GPU를 사용하는 경우 모두에 시드 설정

# cuDNN에서 최적의 성능을 내기 위한 벤치마크 모드 활성화
torch.backends.cudnn.benchmark = True

In [7]:
# 데이터셋 클래스를 정의합니다.
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        """
        ImageDataset 클래스의 초기화 메서드입니다.
        
        :param csv: 이미지 파일 이름과 레이블이 포함된 CSV 파일 경로
        :param path: 이미지 파일이 저장된 디렉토리 경로
        :param transform: 이미지 변환을 위한 Albumentations 변환 함수
        """
        # CSV 파일을 읽어 데이터프레임으로 저장
        self.df = pd.read_csv(csv).values
        self.path = path  # 이미지 파일이 저장된 경로
        self.transform = transform  # 이미지 변환 함수

    def __len__(self):
        """
        데이터셋의 길이를 반환하는 메서드입니다.
        
        :return: 데이터셋의 길이
        """
        return len(self.df)

    def __getitem__(self, idx):
        """
        인덱스에 해당하는 데이터를 반환하는 메서드입니다.
        
        :param idx: 데이터 인덱스
        :return: 변환된 이미지와 해당 레이블
        """
        name, target = self.df[idx]  # 인덱스에 해당하는 이미지 파일 이름과 레이블 가져오기
        img = np.array(Image.open(os.path.join(self.path, name)))  # 이미지 파일 열기
        if self.transform:
            # 이미지에 변환 적용
            img = self.transform(image=img)['image']
        return img, target  # 변환된 이미지와 레이블 반환


In [8]:
# 한 epoch 동안 학습을 수행하는 함수
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()  # 모델을 학습 모드로 전환
    train_loss = 0  # 전체 손실 초기화
    preds_list = []  # 예측값을 저장할 리스트 초기화
    targets_list = []  # 실제 값을 저장할 리스트 초기화

    # 데이터 로더를 tqdm을 이용해 진행률 표시줄과 함께 사용
    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)  # 이미지를 장치(GPU/CPU)로 이동
        targets = targets.to(device)  # 타겟을 장치(GPU/CPU)로 이동

        model.zero_grad(set_to_none=True)  # 모델의 기울기를 초기화

        preds = model(image)  # 모델을 통해 예측값 생성
        loss = loss_fn(preds, targets)  # 예측값과 타겟 간의 손실 계산
        loss.backward()  # 손실에 대한 기울기 역전파
        optimizer.step()  # 모델의 가중치 업데이트

        train_loss += loss.item()  # 손실 누적
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())  # 예측값을 리스트에 추가
        targets_list.extend(targets.detach().cpu().numpy())  # 실제 값을 리스트에 추가

        pbar.set_description(f"Loss: {loss.item():.4f}")  # 진행률 표시줄에 현재 손실 표시

    train_loss /= len(loader)  # 평균 손실 계산
    train_acc = accuracy_score(targets_list, preds_list)  # 정확도 계산
    train_f1 = f1_score(targets_list, preds_list, average='macro')  # F1 점수 계산

    ret = {
        "train_loss": train_loss,  # 평균 손실
        "train_acc": train_acc,  # 정확도
        "train_f1": train_f1,  # F1 점수
    }

    return ret  # 결과 반환


## 3. Hyper-parameters
* 학습 및 추론에 필요한 하이퍼파라미터들을 정의합니다.

#### 이미지 분류에 효율적인 모델들은 주로 CNN(Convolutional Neural Network)과 최근의 Transformer 기반 모델로 나눌 수 있습니다. 각 큰 분류별로 대표적인 모델과 그 특징을 설명하겠습니다.

1. CNN 기반 모델
CNN 기반 모델은 주로 이미지의 공간적 정보를 효과적으로 처리하는데 강점을 가지며, 다양한 크기의 필터를 사용하여 이미지 특징을 추출합니다.

- ResNet (Residual Networks)
모델명: resnet18, resnet34, resnet50, resnet101, resnet152
특징: 깊은 신경망에서 발생할 수 있는 기울기 소실 문제를 해결하기 위해 잔차 연결(residual connection)을 도입. 다양한 깊이의 모델이 존재하며, 각 모델은 이미지 분류에서 매우 높은 성능을 보여줌.
- DenseNet (Densely Connected Networks)
모델명: densenet121, densenet161, densenet169, densenet201
특징: 각 레이어가 이전 모든 레이어의 출력을 입력으로 받는 구조. 파라미터 효율성이 높고, 강력한 특징 추출 능력을 보임.
- EfficientNet
모델명: efficientnet_b0, efficientnet_b1, efficientnet_b2, efficientnet_b3, efficientnet_b4, efficientnet_b5, efficientnet_b6, efficientnet_b7
특징: 모델 크기와 컴퓨팅 자원을 효율적으로 사용하는 네트워크. 성능과 효율성 사이의 균형이 잘 맞음.
- MobileNet
모델명: mobilenetv2_035, mobilenetv2_050, mobilenetv2_075, mobilenetv2_100
특징: 모바일 및 임베디드 기기에서 효율적으로 동작하도록 설계. 경량화된 구조로 빠른 연산 속도를 자랑.
2. Transformer 기반 모델
최근의 Vision Transformer (ViT) 모델은 Transformer 구조를 이미지 분류에 적용한 것으로, 이미지 패치를 시퀀스로 변환하여 처리합니다.

- Vision Transformer (ViT)
모델명: vit_base_patch16_224, vit_base_patch32_224, vit_large_patch16_224, vit_large_patch32_224
특징: Transformer 구조를 기반으로 이미지 패치를 처리. 높은 성능을 보이며, 특히 대규모 데이터셋에서 좋은 성능을 나타냄.
- DeiT (Data-efficient Image Transformers)
모델명: deit_base_patch16_224, deit_small_patch16_224, deit_tiny_patch16_224
특징: 데이터 효율성을 개선한 ViT 변형. 상대적으로 적은 데이터로도 높은 성능을 발휘.
- Swin Transformer (Shifted Window Transformer)
모델명: swin_base_patch4_window7_224, swin_large_patch4_window7_224
특징: 윈도우 기반의 Transformer로, 지역적 특징을 효과적으로 추출. 다양한 스케일에서 좋은 성능을 보임.
3. Hybrid 모델
Hybrid 모델은 CNN과 Transformer의 장점을 결합하여 효율성과 성능을 극대화한 모델입니다.

- CoAtNet (Convolution and self-Attention Network)
모델명: coatnet_0_224, coatnet_1_224, coatnet_2_224
특징: Convolution과 self-Attention 메커니즘을 결합하여 이미지 분류 성능을 극대화. 효율적인 구조로 높은 성능을 보임.
- ConvNeXt
모델명: convnext_tiny, convnext_small, convnext_base, convnext_large
특징: ConvNeXt는 전통적인 CNN 구조를 현대적인 개선을 통해 Transformer와 경쟁할 수 있는 성능을 보임.

추천
- 효율성과 성능을 동시에 고려할 때: EfficientNet 시리즈는 다양한 크기와 성능을 제공하므로, 모바일부터 고성능 서버까지 폭넓게 사용 가능.
- 최신 기술을 활용하고자 할 때: Vision Transformer (ViT)와 DeiT는 최신 Transformer 기술을 기반으로 높은 성능을 보장.
- Hybrid 모델의 성능을 원할 때: CoAtNet과 ConvNeXt는 CNN과 Transformer의 장점을 결합하여 최고의 성능을 제공.
이러한 모델 중에서 선택은 사용자의 요구사항과 환경에 따라 달라질 수 있습니다. 각 모델의 상세한 특성과 벤치마크 결과를 참고하여 최적의 모델을 선택하는 것이 중요합니다.

#### 계속

In [35]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# data config
data_path = '/home/data/'
home_path = '/home/'

# model config
model_name = 'efficientnet_b2' #'convnext_small' # 'resnet34' 'resnet50' 'efficientnet-b0', ...

# training config
img_size = 32
#img_size = 224
LR = 1e-3
EPOCHS = 1
BATCH_SIZE = 32
num_workers = 0

이미지가 기울어져 있거나 뒤집혀 있는 경우, 이러한 변형을 처리할 수 있는 모델과 전처리 방법을 고려해야 합니다. 이미지 변형에 대해 강건한 모델과 전처리 방법을 추천드리겠습니다.

1. 모델 선택
기울어지거나 뒤집힌 이미지를 처리하는 데 있어 강건한 모델로는 다음과 같은 모델들이 있습니다:
- ResNet (Residual Networks)
특징: 잔차 연결을 사용하여 깊은 신경망에서도 효과적으로 학습을 수행. 다양한 변형에 대해 비교적 강건함.
- EfficientNet
특징: 효율적이고 강력한 성능을 제공하며, 데이터 증강을 통해 다양한 이미지 변형에 대해 강건할 수 있음.
- Vision Transformer (ViT)
특징: 이미지 패치를 입력 받아 처리하므로, 각 패치의 상대적 위치에 덜 민감할 수 있음. 기울어짐 뒤집힘에 대한 강건성을 데이터 증강을 통해 강화 가능.
- Swin Transformer
특징: Shifted Window 방식을 사용하여 지역적 특징을 잘 포착. 다양한 변형에 대해 강건한 성능을 보일 수 있음.

In [36]:
bat = 'effi'

### 모델 리스트
all_densenet_models = timm.list_models('*')

aldf = pd.DataFrame(all_densenet_models)
aldf.columns = ['name']

# 'name' 컬럼에서 'bat'로 시작하는 행 필터링
filtered_df = aldf[aldf['name'].str.startswith(bat)]
print(filtered_df)

                         name
201        efficientformer_l1
202        efficientformer_l3
203        efficientformer_l7
204       efficientformerv2_l
205      efficientformerv2_s0
206      efficientformerv2_s1
207      efficientformerv2_s2
208           efficientnet_b0
209     efficientnet_b0_g8_gn
210  efficientnet_b0_g16_evos
211        efficientnet_b0_gn
212           efficientnet_b1
213    efficientnet_b1_pruned
214           efficientnet_b2
215    efficientnet_b2_pruned
216           efficientnet_b3
217     efficientnet_b3_g8_gn
218        efficientnet_b3_gn
219    efficientnet_b3_pruned
220           efficientnet_b4
221           efficientnet_b5
222           efficientnet_b6
223           efficientnet_b7
224           efficientnet_b8
225     efficientnet_cc_b0_4e
226     efficientnet_cc_b0_8e
227     efficientnet_cc_b1_8e
228           efficientnet_el
229    efficientnet_el_pruned
230           efficientnet_em
231           efficientnet_es
232    efficientnet_es_pruned
233       

In [37]:
print(data_path + "data.csv")

/home/data/data.csv


## 4. Load Data
* 학습, 테스트 데이터셋과 로더를 정의합니다.

In [38]:
"""
# augmentation을 위한 transform 코드
trn_transform = A.Compose([
    # 이미지 크기 조정
    A.Resize(height=img_size, width=img_size),
    # images normalization
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    # numpy 이미지나 PIL 이미지를 PyTorch 텐서로 변환
    ToTensorV2(),
])

# test image 변환을 위한 transform 코드
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])
"""

# GPT 추천 
# 이미지 변환을 위한 Albumentations 파이프라인 정의
trn_transform = A.Compose([
    # 45도 범위 내에서 랜덤 회전
    A.Rotate(limit=90, p=1.0),
    # 좌우 뒤집기
    A.HorizontalFlip(p=0.5),
    # 상하 뒤집기
    A.VerticalFlip(p=0.5),
    # 아핀 변환 (translation)
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=0, p=1.0),
    # 이미지 크기 조정
    A.Resize(height=img_size, width=img_size),
    # 이미지 정규화
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    # 이미지를 PyTorch 텐서로 변환
    ToTensorV2(),
])

# test image 변환을 위한 transform 코드
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

In [40]:
# Dataset 정의
trn_dataset = ImageDataset(
    data_path + "train.csv",
    data_path + "train/",
    transform=trn_transform
)
tst_dataset = ImageDataset(
    data_path + "sample_submission.csv",
    data_path + "test/",
    transform=tst_transform
)
print(len(trn_dataset), len(tst_dataset))

1570 3140


In [41]:
# DataLoader 정의
trn_loader = DataLoader(
    trn_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
)
tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

## 5. Train Model
* 모델을 로드하고, 학습을 진행합니다.

In [42]:
# load model
model = timm.create_model(
    model_name,
    pretrained=False,  # RuntimeError: No pretrained weights exist for coatnet_0_224. Use `pretrained=False` for random init.
    num_classes=17
).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)

In [43]:
%%time 
#baseline 14.8s | 
#Loss: 2.5325: 100%|██████████| 50/50 [00:06<00:00,  7.65it/s]
#train_loss: 2.4731
#train_acc: 0.2554
#train_f1: 0.2238
#epoch: 0.0000

# 이게 기본값인가?
#Loss: 1.8296: 100%|██████████| 50/50 [00:09<00:00,  5.28it/s]
#train_loss: 1.6996
#train_acc: 0.5682
#train_f1: 0.5234
#epoch: 0.0000
#CPU times: user 22.1 s, sys: 624 ms, total: 22.7 s
#Wall time: 9.47 s

for epoch in range(EPOCHS):
    ret = train_one_epoch(trn_loader, model, optimizer, loss_fn, device=device)
    ret['epoch'] = epoch

    log = ""
    for k, v in ret.items():
      log += f"{k}: {v:.4f}\n"
    print(log)

Loss: 6.6409: 100%|██████████| 50/50 [00:11<00:00,  4.28it/s]

train_loss: 5.2883
train_acc: 0.1102
train_f1: 0.1017
epoch: 0.0000

CPU times: user 29.4 s, sys: 8.74 s, total: 38.1 s
Wall time: 11.7 s


In [44]:
for epoch in range(EPOCHS):
    ret = train_one_epoch(trn_loader, model, optimizer, loss_fn, device=device)
    ret['epoch'] = epoch

    log = ""
    for k, v in ret.items():
      log += f"{k}: {v:.4f}\n"
    print(log)

Loss: 5.9813: 100%|██████████| 50/50 [00:10<00:00,  4.82it/s]

train_loss: 2.8855
train_acc: 0.1522
train_f1: 0.1284
epoch: 0.0000



# 6. Inference & Save File
* 테스트 이미지에 대한 추론을 진행하고, 결과 파일을 저장합니다.

In [45]:
preds_list = []

model.eval()  # 모델을 평가 모드로 전환
for image, _ in tst_loader:  #tqdm(tst_loader): # 테스트 데이터 로더에서 이미지 가져오기
    image = image.to(device)  # 이미지를 GPU로 전송

    with torch.no_grad():  # 기울기 계산 비활성화
        preds = model(image)  # 예측 수행
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())  # 예측 결과 처리 및 저장

In [18]:
### test 데이터셋 분류 번호 붙이기
pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list

In [19]:
sample_submission_df = pd.read_csv(data_path + "sample_submission.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

In [50]:
from datetime import datetime
import pytz

# 현재 UTC 시간
utc_now = datetime.utcnow()

# 한국 시간대
kst = pytz.timezone('Asia/Seoul')

# 한국 시간으로 변환
kst_now = utc_now.astimezone(kst)

# 원하는 포맷으로 시간 출력
current_time_kst = kst_now.strftime("%m%d_%H%M")
print("현재 한국 시간:", current_time_kst)

현재 한국 시간: 0730_1720


In [21]:
coding = input("변경사항: ")

In [22]:
if coding != "" :
    pred_df.to_csv(home_path + "pred" + current_time_kst + "_" + coding + ".csv", index=False)
else:
    print("pred 파일로 저장하지 않음.")

In [23]:
pred_df.head()

,ID,target
0,0008fdb22ddce0ce.jpg,2
1,00091bffdffd83de.jpg,3
2,00396fbc1f6cc21d.jpg,5
3,00471f8038d9c4b6.jpg,3
4,00901f504008d884.jpg,2
